In [1]:
import pandas as pd
import numpy as np

----

# Preprocessing

In [2]:
df = pd.read_csv('UserBehavior.csv',names=['uid','gid','cid','action','time'])
df.head()

,uid,gid,cid,action,time
0,1,2268318,2520377,pv,1511544070
1,1,2333346,2520771,pv,1511561733
2,1,2576651,149192,pv,1511572885
3,1,3830808,4181361,pv,1511593493
4,1,4365585,2520377,pv,1511596146


In [3]:
len(df)

100150807

In [4]:
df.iloc[:len(df)//4].to_csv('df1.csv', index=0)
df.iloc[len(df)//4:len(df)//2].to_csv('df2.csv',index=0)
df.iloc[len(df)//2:len(df)//4*3].to_csv('df3.csv',index=0)
df.iloc[len(df)//4*3:].to_csv('df4.csv',index=0) #seperate df into 4 parts

In [5]:
def process(data=0, i=0): #timestamp to datetime
    df = data
    df['time'] = [pd.Timestamp(i, unit='s') for i in df['time']]
    df.to_csv('df'+str(i)+'.csv', index=0)
    return df.head()

In [7]:
del df #processing df1~4
for i in range(1,5):
    df = pd.read_csv('df'+str(i)+'.csv')
    process(data=df,i=i)
    del df

In [3]:
df = pd.read_csv('df1.csv') #concat df1~4
for i in range(2,5):
    ndf = pd.read_csv('df'+str(i)+'.csv')
    df = pd.concat([df,ndf])
    del ndf
    print(i)
df.to_csv('processed.csv',index=0)

2
3
4


In [2]:
df = pd.read_csv('processed.csv') #read processed df
df = df.drop('Unnamed: 0',axis=1)
df.info()

In [6]:
df.head()

,uid,gid,cid,action,time
0,1,2268318,2520377,pv,2017-11-24 17:21:10
1,1,2333346,2520771,pv,2017-11-24 22:15:33
2,1,2576651,149192,pv,2017-11-25 01:21:25
3,1,3830808,4181361,pv,2017-11-25 07:04:53
4,1,4365585,2520377,pv,2017-11-25 07:49:06


In [7]:
df.isnull().sum() #check null

uid       0
gid       0
cid       0
action    0
time      0
dtype: int64

----

# Stats 

In [2]:
def select_time(data=None, ranging=None): #data ranging form a to b datetime
    return data.loc[(data['time']>=ranging[0]) & (data['time']<=ranging[1])]

In [3]:
def main(test=True):
    if test == True:
        df = pd.read_csv('df1.csv')
        df = df.head(10240)
    else:
        df = pd.read_csv('processed.csv') #read processed df
        df = df.drop('Unnamed: 0', axis=1)
        print(df.info())
    df['time'] = pd.to_datetime(df.time)
    #select data by time range
    ranging = input('selecting range of time:2017.11.24-00:16:16-2017.11.25-05:05:06').split('-')
    print(ranging)
    d0 = [int(i) for i in ranging[0].split('.')]
    t0 = [int(i) for i in ranging[1].split(':')]
    d1 = [int(i) for i in ranging[2].split('.')]
    t1 = [int(i) for i in ranging[3].split(':')]
    time0 = pd.Timestamp(d0[0],d0[1],d0[2],t0[0],t0[1],t0[2])
    time1 = pd.Timestamp(d1[0],d1[1],d1[2],t1[0],t1[1],t1[2])
    df = select_time(data=df, ranging=[time0, time1])
    #simple stasts 
    print('用户数：', len(df.groupby('uid').agg('count')))
    print('商品数：', len(df.groupby('gid').agg('count')))
    print('用户行为数：', len(df.groupby('action').agg('count')))
    print('总条目数：', len(df))
    #first getting required gid, then extracting corresponding cid
    pvtop = df.set_index('action').loc['pv'].groupby('gid').agg('count').sort_values('time',ascending=False).head(10).index
    buytop = df.set_index('action').loc['buy'].groupby('gid').agg('count').sort_values('time',ascending=False).head(10).index
    print('浏览数前10商品及其类别:',df.groupby('gid').agg('mean')['cid'].astype('int').loc[pvtop])
    print('购买数前10商品及其类别:',df.groupby('gid').agg('mean')['cid'].astype('int').loc[buytop])
    

In [4]:
main(False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100150807 entries, 0 to 100150806
Data columns (total 5 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   uid     int64 
 1   gid     int64 
 2   cid     int64 
 3   action  object
 4   time    object
dtypes: int64(3), object(2)
memory usage: 3.7+ GB
None
用户数： 987983
商品数： 3982878
用户行为数： 4
总条目数： 88264377
浏览数前10商品及其类别: gid
812879     4756105
3845720    3607361
138964     3607361
2331370    3607361
3371523    1859277
2032668    1080785
2367945    1859277
1591862    1320293
2338453    3607361
4211339    4756105
Name: cid, dtype: int32
购买数前10商品及其类别: gid
3122135    1516409
3031354    2735466
3964583     800581
2560262    2735466
2964774    1265358
1116492    2297500
1910706    1516409
740947      904454
1415828    2832009
4443059     570735
Name: cid, dtype: int32
